In [1]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='train')
sentences = data['data']
labels = data['target']


In [2]:
from datasets import load_dataset
dataset = load_dataset("ag_news")


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import random

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Parameters
vocab_size = 10000
embedding_dim = 300
max_length = 50
num_classes = 20

# Revised Model Architecture
input_sentence = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(input_sentence)
bi_lstm = Bidirectional(LSTM(256, return_sequences=True))(embedding)
pooling = GlobalMaxPooling1D()(bi_lstm)  # Add global max pooling to reduce dimensions
dropout = Dropout(0.5)(pooling)
dense = Dense(256, activation='relu')(dropout)
context_output = Dense(num_classes, activation='softmax')(dense)

context_model = Model(inputs=input_sentence, outputs=context_output)
context_model.compile(optimizer=Adam(learning_rate=1e-4), 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])

# Print model summary to verify
context_model.summary()

# Load 20 newsgroups dataset
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Fetch dataset
data = fetch_20newsgroups(subset='train')
sentences = data['data']
labels = data['target']

# Tokenize the sentences
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Convert labels to categorical
categorical_labels = to_categorical(labels, num_classes=num_classes)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, 
    categorical_labels, 
    test_size=0.2, 
    random_state=42
)

# Train the model
history = context_model.fit(
    X_train, y_train, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = context_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 50, 300)           3000000   
                                                                 
 bidirectional_5 (Bidirecti  (None, 50, 512)           1140736   
 onal)                                                           
                                                                 
 global_max_pooling1d (Glob  (None, 512)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               1313

In [29]:
context_model.summary()  # Inspect input shape of the model


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 50, 300)           3000000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 50, 512)           1140736   
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 50, 512)           0         
                                                                 
 dense_8 (Dense)             (None, 50, 256)           131328    
                                                                 
 dense_9 (Dense)             (None, 50, 20)            5140      
                                                           

In [35]:
from datasets import load_dataset

# Load PAWS dataset
paws = load_dataset("paws", "labeled_final")
print(paws['train'][1])  # Check the structure of the first entry

# Get the first sentence pair and label from the PAWS dataset
sentence1 = paws['train'][1]['sentence1']
sentence2 = paws['train'][1]['sentence2']
paws_label = paws['train'][1]['label']


{'id': 2, 'sentence1': 'The NBA season of 1975 -- 76 was the 30th season of the National Basketball Association .', 'sentence2': 'The 1975 -- 76 season of the National Basketball Association was the 30th season of the NBA .', 'label': 1}


In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset

# Load PAWS dataset
paws = load_dataset("paws", "labeled_final")

# Parameters
max_length = 50  # Consistent with model training
vocab_size = 10000  # Consistent with model training

# Initialize tokenizer
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(paws['train']['sentence1'] + paws['train']['sentence2'])

def prepare_input(sentences, tokenizer, max_length):
    """Prepare input sequences for the model"""
    sequences = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    return padded

def predict_context(model, sentences, tokenizer, max_length):
    """Predict the context of sentences"""
    padded_input = prepare_input(sentences, tokenizer, max_length)
    prediction = model.predict(padded_input)
    return prediction.argmax(axis=1)  # Get the class index with the highest probability

# Example usage with the first PAWS example
sentence1 = paws['train'][1]['sentence1']
sentence2 = paws['train'][1]['sentence2']
paws_label = paws['train'][1]['label']

# Predict context for both sentences
contexts = predict_context(context_model, [sentence1, sentence2], tokenizer, max_length)

# Check similarity based on predicted contexts
similarity = int(contexts[0] == contexts[1])  # Compare predicted context values

# Compare the predicted similarity with the PAWS label
is_correct = (similarity == paws_label)

print(f"Sentence 1: {sentence1}")
print(f"Sentence 2: {sentence2}")
print(f"PAWS label: {paws_label}")
print(f"Predicted contexts: {contexts}")
print(f"Predicted similarity: {'similar' if similarity else 'not similar'}")
print(f"Correct prediction: {'Yes' if is_correct else 'No'}")

1/1 [==============================] - 0s 29ms/step
Sentence 1: The NBA season of 1975 -- 76 was the 30th season of the National Basketball Association .
Sentence 2: The 1975 -- 76 season of the National Basketball Association was the 30th season of the NBA .
PAWS label: 1
Predicted contexts: [13 13]
Predicted similarity: similar
Correct prediction: Yes


In [38]:
def evaluate_accuracy(model, dataset, tokenizer, max_length):
    """Evaluate the accuracy of the model on the PAWS dataset."""
    correct_predictions = 0
    total_predictions = len(dataset['sentence1'])

    similar_pairs = []
    dissimilar_pairs = []

    for i in range(total_predictions):
        sentence1 = dataset[i]['sentence1']
        sentence2 = dataset[i]['sentence2']
        paws_label = dataset[i]['label']

        # Predict similarity for this pair
        contexts = predict_context(model, [sentence1, sentence2], tokenizer, max_length)

        # Check similarity based on predicted contexts
        similarity = int(contexts[0] == contexts[1])

        # Increment correct predictions if the predicted similarity matches the true label
        if similarity == paws_label:
            correct_predictions += 1

        # Store the sentence pairs based on the predicted similarity
        if similarity == 1:
            similar_pairs.append((sentence1, sentence2, paws_label, similarity))
        else:
            dissimilar_pairs.append((sentence1, sentence2, paws_label, similarity))

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions

    # Print 5 similar and 5 dissimilar sentence pairs
    print("\n5 Similar Sentence Pairs:")
    for pair in random.sample(similar_pairs, 5):  # Randomly select 5 similar pairs
        print(f"Sentence 1: {pair[0]}")
        print(f"Sentence 2: {pair[1]}")
        print(f"True Label: {pair[2]}, Predicted Similarity: {pair[3]}")
        print()

    print("\n5 Dissimilar Sentence Pairs:")
    for pair in random.sample(dissimilar_pairs, 5):  # Randomly select 5 dissimilar pairs
        print(f"Sentence 1: {pair[0]}")
        print(f"Sentence 2: {pair[1]}")
        print(f"True Label: {pair[2]}, Predicted Similarity: {pair[3]}")
        print()

    return accuracy

# Example usage:
accuracy = evaluate_accuracy(context_model, paws['test'], tokenizer, max_length)
print(f"Model accuracy on PAWS test dataset: {accuracy * 100:.2f}%")


1/1 [==============================] - 0s 34ms/step

5 Similar Sentence Pairs:
Sentence 1: The tiny island is from northwest to southeast meters long and up to meters wide and roughly area .
Sentence 2: The tiny island is meters wide from northwest to southeast , and up to meters long , and about in area .
True Label: 0, Predicted Similarity: 1

Sentence 1: In 1985 , Havelsan was separated from the foreign shareholders and incorporated as a national company with a share of 98 % owned by the Turkish Armed Forces Foundation .
Sentence 2: In 1985 , Havelsan was separated from the foreign shareholders and founded as a national company with a share of 98 % owned by the Turkish Armed Forces Foundation .
True Label: 1, Predicted Similarity: 1

Sentence 1: Assembled by producer Mick Walsh , the group also included singers Melody Washington and Stevie Vincent .
Sentence 2: Compiled by producer Mick Walsh , the group also included singers Melody Washington and Stevie Vincent .
True Label: 1, Pre